In [1]:
import pandas as pd
import aiohttp
import asyncio
import re
from tqdm import tqdm
import warnings
import os
warnings.filterwarnings("ignore")

C:\Users\Eugene\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ETHERSCAN_API_KEY = 'GGJU543MXKGFSRTRIGBKEDG8F4CR637M5S' # ввести свой ключ в сети эфириум
BSCSCAN_API_KEY = 'F3ZA4NB4MEGXTWI3FNKZG73RY5CF6ZUU3N' # ввести свой ключ в сети эфириум

In [3]:
async def get_contract_source_code(address, provider) -> dict:
    '''
    функция, чтобы достать информацию о транзакции
    address - адрес транзакции
    provider - тип платформы транзакции (напр. eth)
    '''
    source_code: dict = {}
    if provider == "trx":
        pass
    else:
        _headers = {
            "Connection": "keep-alive",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept": "*/*",
            "User-Agent": "PostmanRuntime/7.34.0"
        }
        api_key = ETHERSCAN_API_KEY
        base_url = "https://api.etherscan.io/api?module=contract&action=getsourcecode"
        
        if provider == "bsc":
            _headers = {
                "Connection": "keep-alive",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept": "*/*",
            }
            api_key = BSCSCAN_API_KEY
            base_url = "https://api.bscscan.com/api?module=contract&action=getsourcecode"
        
        full_url = f"{base_url}&address={address}&apikey={api_key}"

        async with aiohttp.ClientSession() as session:
            async with session.get(full_url, headers=_headers) as response:
                data = await response.json()
        #print(data)
        source_code = data["result"][0]
    
    return source_code

# Читаем датасет

In [4]:
df = pd.read_csv('C:\\Users\\Eugene\\Desktop\\add_bsc_eth_scam_contracts.csv')
df['transferFrom'] = ''
df['transfer'] = ''
df['_transfer'] = ''
df['_transferFrom'] = ''
df['beforeTokenTransfer'] = ''
df['_beforeTokenTransfer'] = ''
df_uniq = df.drop_duplicates('transact').reset_index().drop(columns=['index'])
df_uniq

,transact,type,transferFrom,transfer,_transfer,_transferFrom,beforeTokenTransfer,_beforeTokenTransfer
0,0x8105ed8a3be363e32c91a0904cd59e0f843d7123,eth,,,,,,
1,0xb16d3ed603d62b125c6bd45519eda40829549489,eth,,,,,,
2,0xa5978a61a90ae7217c60c7b275343ce244a053cc,eth,,,,,,
3,0x52e8ebf98ef95d373d29041182830bf13c52588f,eth,,,,,,
4,0x91e64f39c1fe14492e8fdf5a8b0f305bd218c8a1,eth,,,,,,
5,0xe460f3e0adc58357ea5e3ac1d839611dee826354,eth,,,,,,
6,0xb7c06096b7cf4622ae0da4ebc929649463c8692f,eth,,,,,,


### Регулярки для вычленения необходимого кода

In [5]:
reForTransferFrom = '(function transferFrom){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
_reForTransferFrom = '(function _transferFrom){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
reForTransfer = '(function transfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
_reForTransfer = '(function _transfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?{([^\}]+[^\{]*)+'
reForBeforeTokenTransfer = '(function beforeTokenTransfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
_reForBeforeTokenTransfer = '(function _beforeTokenTransfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'

In [6]:
def extractTransfer(allStr):
    '''
    функция, чтобы достать искомой части из source_code
    '''
    str_in_array = allStr.split(sep='}', maxsplit=-1)
    finalTransferFrom = str_in_array[0] + '}'
    remaining_part = str_in_array
    remaining_part.pop(0)
    
    for i in range(1, len(str_in_array)):
        '''print(str_in_array)
        print('///////////////////////////////////')'''
        if 'function' not in str_in_array[0]:
            finalTransferFrom += str_in_array[0]
            finalTransferFrom += '}'
            remaining_part.pop(0)
        else:
            remaining_part.pop(0)
            break
    
    remaining_source_code = '' 
    
    for i in range(len(remaining_part)):
        remaining_source_code += remaining_part[i]
        remaining_source_code += '}'
    #print(finalTransferFrom)
    #print('///////////////////////////////////')
    return finalTransferFrom, remaining_source_code

In [11]:
async def extractFunction(transFunc, transRegex):
    '''
    функция для получения source_code транзакции и парсингом требуемой функции из него
    transFunc - функция для парсинга
    transRegex - регулярное выражения для этой функции
    '''
    for i in tqdm(range(df.shape[0])):
        if df.loc[i, 'type'] == 'bsc':
            source_code = await get_contract_source_code(df['transact'][i], 'bsc')
        else:
            source_code = await get_contract_source_code(df['transact'][i], 'eth')
        source_code = source_code['SourceCode']
        tr = ''
        founded_funcs = []
        if source_code != '':
            num_funcs = len(re.findall(transFunc, source_code))
            for j in range(num_funcs):
                num_funcs = len(re.findall(transFunc, source_code))
                receivedFunc = re.search(transRegex, source_code)
                if receivedFunc is not None:
                    allStr = receivedFunc.group(0)
                    tr, source_code = extractTransfer(allStr)
                    founded_funcs.append(tr)
                else:
                    tr = ''
                    break
                    
        maxstr = ''
        maxlen = float('-inf')
        for found in founded_funcs:
            if len(found) > maxlen:
                maxstr = found
                maxlen = len(found)
        # удаление комментариев
        maxstr = re.sub(r'//.*', '', maxstr)
        maxstr = re.sub(r'/\*[\s\S]*?\*/', '', maxstr)
        df.loc[i, transFunc] = maxstr

In [8]:
await extractFunction('transferFrom', reForTransferFrom)
await extractFunction('transfer', reForTransfer)
await extractFunction('_transferFrom', _reForTransferFrom)
await extractFunction('_transfer', _reForTransfer)
await extractFunction('beforeTokenTransfer', reForBeforeTokenTransfer)
await extractFunction('_beforeTokenTransfer', _reForBeforeTokenTransfer)

100%|██████████| 7/7 [00:04<00:00,  1.45it/s]


In [10]:
df.to_csv('eth_bsc_done_scam_contracts.csv', index=False)

In [9]:
df

,transact,type,transferFrom,transfer,_transfer,_transferFrom,beforeTokenTransfer,_beforeTokenTransfer
0,0x8105ed8a3be363e32c91a0904cd59e0f843d7123,eth,"function transferFrom(address from, address to...","function transfer(address to, uint tokens) pub...",,,,
1,0xb16d3ed603d62b125c6bd45519eda40829549489,eth,"function transferFrom(address _from, address _...","function transfer(address _to, uint256 _value)...",,,,
2,0xa5978a61a90ae7217c60c7b275343ce244a053cc,eth,,,,,,
3,0x52e8ebf98ef95d373d29041182830bf13c52588f,eth,"function transferFrom(address sender, address ...","function transfer(address recipient, uint256 a...",,,,"function _beforeTokenTransfer(address from, ad..."
4,0x91e64f39c1fe14492e8fdf5a8b0f305bd218c8a1,eth,"function transferFrom(address _from, address _...","function transfer(address to, uint value) publ...",,,,
5,0xe460f3e0adc58357ea5e3ac1d839611dee826354,eth,"function transferFrom(address sender, address ...","function transfer(address recipient, uint256 a...",,,,"function _beforeTokenTransfer(address from, ad..."
6,0xb7c06096b7cf4622ae0da4ebc929649463c8692f,eth,"function transferFrom(address sender, address ...","function transfer(address recipient, uint256 a...",,,,"function _beforeTokenTransfer(address from, ad..."


# Читаем файл транзакции

In [76]:
save_directory = 'C:\\Users\\Eugene\\Desktop\\' # Директория для сохранения датафрейма
file_to_save = 'transfers_from_files.csv' # файл для сохранения датафрейма
#file_to_read = '0x8ed9c7e4d8dfe480584cc7ef45742ac302ba27d7.sol' # файл для прочтения
file_to_read= '0x05cb447e5bd814e728a1cd79eedb79b2e011685e.sol'

In [77]:
if not os.path.exists(save_directory + file_to_save):
    df = pd.DataFrame(columns=['transact', 'transferFrom', '_transfer', '_transferFrom'])
    df.to_csv(save_directory + file_to_save, index=False)

In [78]:
df = pd.read_csv(save_directory + file_to_save)
index = df.shape[0]
f = open(save_directory + file_to_read, "r")
file = f.read()

In [81]:
df

,transact,transferFrom,_transfer,_transferFrom,transfer
0,0x8ed9c7e4d8dfe480584cc7ef45742ac302ba27d7,function transferFrom(\n\n address send...,"function _transfer(\n\n address sender,...",NaN,"function transfer(address recipient, uint256 a..."
1,0x05cb447e5bd814e728a1cd79eedb79b2e011685e,NaN,NaN,NaN,NaN


### добавление адреса транзакции работает, если название файла соответстувет шаблону transaction.sol

In [80]:
df.loc[index, 'transact'] = str(file_to_read.split('.')[0])

In [82]:
async def extractFunctionFile(source_code, transFunc, transRegex):
    '''
    функция для получения source_code транзакции и парсингом требуемой функции из него
    source_code - source_code из файла
    transFunc - функция для парсинга
    transRegex - регулярное выражения для этой функции
    '''
    tr = ''
    if file != '':
        receivedFunc = re.search(transRegex, source_code)
        if receivedFunc is not None:
            allStr = receivedFunc.group(0)
            tr = extractTransfer(allStr)
        else:
            tr = ''
    df.loc[index, transFunc] = tr

In [83]:
await extractFunctionFile(file, 'transfer', reForTransfer)
await extractFunctionFile(file, 'transferFrom', reForTransferFrom)
await extractFunctionFile(file, '_transfer', _reForTransfer)
await extractFunctionFile(file, '_transferFrom', _reForTransferFrom)

In [84]:
df.to_csv(save_directory + file_to_save, index=False)